In [5]:
# money
import os
import json
import pandas as pd
import numpy as np
import re
from scipy import stats
from scipy.signal import welch


def load_config(config_path='config.json'):
    """
    Load configuration parameters from a JSON file.
    """
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Configuration file '{config_path}' not found.")
    
    with open(config_path, 'r') as f:
        config = json.load(f)
    
    if 'base_dir' not in config:
        raise KeyError("Missing required configuration key: 'base_dir'")
    
    return config


def compute_statistical_features(data_array):
    """
    Compute statistical features from a data array.
    """
    features = {}
    try:
        features['entropy'] = stats.entropy(np.abs(data_array))
        features['skewness'] = stats.skew(data_array)
        features['interquartile_range'] = np.percentile(data_array, 75) - np.percentile(data_array, 25)
        features['kurtosis'] = stats.kurtosis(data_array)
        features['percentile_75'] = np.percentile(data_array, 75)
        features['range'] = np.ptp(data_array)
        features['maximum'] = np.max(data_array)
        features['median'] = np.median(data_array)
        features['percentile_90'] = np.percentile(data_array, 90)
        features['mean_absolute_deviation'] = np.mean(np.abs(data_array - np.mean(data_array)))
    except Exception as e:
        print(f"Error computing statistical features: {e}")
        features = {key: np.nan for key in [
            'entropy', 'skewness', 'interquartile_range', 'kurtosis',
            'percentile_75', 'range', 'maximum', 'median',
            'percentile_90', 'mean_absolute_deviation']}
    return features


def process_channel(data, decimation_factor, processing_method):
    """
    Process a single channel's data and return computed features.
    """
    try:
        if len(data) == 0:
            print("No data found in the channel.")
            return None

        # Remove NaN values
        data = data.dropna()

        # Detrend signal
        signal = data.values
        signal = signal - np.mean(signal)  # Remove DC component

        if processing_method.lower() == 'welch':
            frequencies, power_spectrum = welch(signal, fs=125e6 / decimation_factor, nperseg=1024)
            data_to_analyze = power_spectrum
        elif processing_method.lower() == 'histogram':
            hist_counts, _ = np.histogram(signal, bins=50, density=True)
            data_to_analyze = hist_counts
        else:
            raise ValueError(f"Invalid processing method: {processing_method}")
        
        # Compute statistical features
        features = compute_statistical_features(data_to_analyze)
        return features
    
    except Exception as e:
        print(f"Error processing channel data: {e}")
        return None


def process_session(file_path, decimation_factor, processing_method):
    """
    Process a single session file, focusing only on voltage_ch1.
    """
    try:
        session_match = re.search(r'session_(\d+)\.csv', os.path.basename(file_path))
        if not session_match:
            print(f"Invalid file format: {file_path}")
            return None
        
        session_number = int(session_match.group(1))

        # Read data
        df = pd.read_csv(file_path)
        if 'voltage_ch1' not in df.columns:
            print(f"Missing 'voltage_ch1' in file: {file_path}")
            return None

        # Process voltage_ch1
        features = process_channel(df['voltage_ch1'], decimation_factor, processing_method)
        if features is None:
            print(f"Failed to process voltage_ch1 for session {session_number}")
            return None
        
        features['session'] = session_number
        features['timestamp'] = df['timestamp'].iloc[0] if 'timestamp' in df.columns else None

        return features

    except Exception as e:
        print(f"Error processing session {file_path}: {e}")
        return None


def main():
    try:
        config = load_config('config.json')
    except Exception as e:
        print(f"Configuration Error: {e}")
        return
    
    base_dir = config['base_dir']
    decimation_factor = config.get('decimation_factor', 1024)
    processing_method = 'welch'

    summary_data = []

    for root, _, files in os.walk(base_dir):
        for file_name in files:
            if file_name.endswith('.csv') and re.match(r'session_\d+\.csv', file_name):
                file_path = os.path.join(root, file_name)
                print(f"Processing file: {file_path}")
                session_data = process_session(file_path, decimation_factor, processing_method)
                if session_data:
                    session_data['device_label'] = os.path.basename(os.path.dirname(root))
                    #session_data['date'] = os.path.basename(root)
                    summary_data.append(session_data)

    if summary_data:
        summary_df = pd.DataFrame(summary_data)
        output_path = os.path.join(base_dir, 'summary_features.csv')
        summary_df.to_csv(output_path, index=False)
        print(f"Summary features saved to {output_path}")
    else:
        print("No data processed.")


if __name__ == "__main__":
    main()


Processing file: /Volumes/One Touch/data_scpi/empty/2024-12-02/session_001.csv
Processing file: /Volumes/One Touch/data_scpi/empty/2024-12-02/session_002.csv
Processing file: /Volumes/One Touch/data_scpi/empty/2024-12-02/session_003.csv
Processing file: /Volumes/One Touch/data_scpi/empty/2024-12-02/session_004.csv
Processing file: /Volumes/One Touch/data_scpi/empty/2024-12-02/session_005.csv
Processing file: /Volumes/One Touch/data_scpi/empty/2024-12-02/session_006.csv
Processing file: /Volumes/One Touch/data_scpi/empty/2024-12-02/session_007.csv
Processing file: /Volumes/One Touch/data_scpi/empty/2024-12-02/session_008.csv
Processing file: /Volumes/One Touch/data_scpi/empty/2024-12-02/session_009.csv
Processing file: /Volumes/One Touch/data_scpi/empty/2024-12-02/session_010.csv
Processing file: /Volumes/One Touch/data_scpi/empty/2024-12-02/session_011.csv
Processing file: /Volumes/One Touch/data_scpi/empty/2024-12-02/session_012.csv
Processing file: /Volumes/One Touch/data_scpi/empty/

# signal processing types
## single folder one by one with config

In [3]:
import os
import json
import pandas as pd
import numpy as np
import re
from scipy import stats
from scipy.signal import welch

# ---------------------------
# Configuration and Setup
# ---------------------------

def load_config(config_path='config.json'):
    """
    Load configuration parameters from a JSON file.
    """
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Configuration file '{config_path}' not found.")
    
    with open(config_path, 'r') as f:
        config = json.load(f)
    
    required_keys = ['base_dir', 'device_label', 'processing_method']
    for key in required_keys:
        if key not in config:
            raise KeyError(f"Missing required configuration key: '{key}'")
    
    if config['processing_method'].lower() not in ['fft', 'welch', 'bd']:
        raise ValueError(f"Invalid processing_method '{config['processing_method']}'. Must be 'fft', 'welch', or 'bd'.")
    
    return config

# ---------------------------
# Statistical Feature Computation
# ---------------------------

def compute_statistical_features(power_dbm):
    """
    Compute the required statistical features from power_dbm.
    """
    features = {}
    try:
        features['entropy'] = stats.entropy(np.abs(power_dbm))
        features['skewness'] = stats.skew(power_dbm)
        features['interquartile_range'] = np.percentile(power_dbm, 75) - np.percentile(power_dbm, 25)
        features['kurtosis'] = stats.kurtosis(power_dbm)
        features['percentile_75'] = np.percentile(power_dbm, 75)
        features['range'] = np.ptp(power_dbm)
        features['maximum'] = np.max(power_dbm)
        features['median'] = np.median(power_dbm)
        features['percentile_90'] = np.percentile(power_dbm, 90)
        features['mean_absolute_deviation'] = np.mean(np.abs(power_dbm - np.mean(power_dbm)))
    except Exception as e:
        print(f"Error computing statistical features: {e}")
        features = {key: np.nan for key in ['entropy', 'skewness', 'interquartile_range', 'kurtosis',
                                           'percentile_75', 'range', 'maximum', 'median',
                                           'percentile_90', 'mean_absolute_deviation']}
    return features

# ---------------------------
# Session Processing
# ---------------------------

def process_session(ch1_filepath, ch2_filepath, decimation_factor, processing_method):
    """
    Process a single session by computing features for CH1 and CH2 using the specified processing method.
    Returns a dictionary with all required information.
    """
    session_data = {}
    try:
        # Extract session number from filename
        session_match = re.search(r'session_(\d{3})_CH1\.csv', os.path.basename(ch1_filepath))
        if not session_match:
            print(f"Invalid CH1 filename format: {ch1_filepath}")
            return None
        session_number = int(session_match.group(1))
        session_data['session'] = session_number
        
        # Extract date from filepath
        # Assuming the path is base_dir/device_label/YYYY-MM-DD/session_###_CHx.csv
        date_str = os.path.basename(os.path.dirname(ch1_filepath))
        session_data['date'] = date_str
        
        # Read CH1 data
        df_ch1 = pd.read_csv(ch1_filepath)
        if df_ch1.empty or 'voltage' not in df_ch1.columns:
            print(f"No data or missing 'voltage' column in {ch1_filepath}")
            return None
        
        # Handle NaN values in CH1
        if df_ch1['voltage'].isnull().any():
            print(f"NaN values found in {ch1_filepath}. They will be ignored.")
            df_ch1 = df_ch1.dropna(subset=['voltage'])
            if df_ch1.empty:
                print(f"All data is NaN in {ch1_filepath}. Skipping session.")
                return None
        
        # Get the first timestamp as session timestamp
        timestamp_str = df_ch1['timestamp'].iloc[0]
        session_data['timestamp'] = timestamp_str
        
        # Process CH1
        signal_ch1 = df_ch1['voltage'].values
        signal_ch1 = signal_ch1 - np.mean(signal_ch1)  # Remove DC component

        if processing_method.lower() == 'welch':
            frequencies_ch1, power_spectrum_ch1 = welch(signal_ch1, fs=125e6 / decimation_factor, nperseg=1024)
        elif processing_method.lower() == 'fft':
            # Using FFT
            N = len(signal_ch1)
            frequencies_ch1 = np.fft.fftfreq(N, d=1/(125e6 / decimation_factor))
            power_spectrum_ch1 = np.abs(np.fft.fft(signal_ch1))**2 / N
        elif processing_method.lower() == 'bd':
            # Placeholder for Blind Deconvolution
            # Implement blind deconvolution here
            # For demonstration, we'll use the same as Welch
            frequencies_ch1, power_spectrum_ch1 = welch(signal_ch1, fs=125e6 / decimation_factor, nperseg=1024)
        else:
            raise ValueError(f"Invalid processing method: {processing_method}")
        
        # Avoid log of zero by replacing zeros with a very small number
        power_spectrum_ch1 = np.where(power_spectrum_ch1 == 0, 1e-12, power_spectrum_ch1)
        power_dbm_ch1 = 10 * np.log10(power_spectrum_ch1 / 50)  # Assuming 50 ohm impedance
        mask_ch1 = (frequencies_ch1 >= 10e3) & (frequencies_ch1 <= 100e3)
        filtered_power_dbm_ch1 = power_dbm_ch1[mask_ch1]
        features_ch1 = compute_statistical_features(filtered_power_dbm_ch1)
        
        # Add CH1 features to session data
        for key, value in features_ch1.items():
            session_data[f'ch1_{key}'] = value
        
        # Read CH2 data
        df_ch2 = pd.read_csv(ch2_filepath)
        if df_ch2.empty or 'voltage' not in df_ch2.columns:
            print(f"No data or missing 'voltage' column in {ch2_filepath}")
            return None
        
        # Handle NaN values in CH2
        if df_ch2['voltage'].isnull().any():
            print(f"NaN values found in {ch2_filepath}. They will be ignored.")
            df_ch2 = df_ch2.dropna(subset=['voltage'])
            if df_ch2.empty:
                print(f"All data is NaN in {ch2_filepath}. Skipping session.")
                return None
        
        # Process CH2
        signal_ch2 = df_ch2['voltage'].values
        signal_ch2 = signal_ch2 - np.mean(signal_ch2)  # Remove DC component

        if processing_method.lower() == 'welch':
            frequencies_ch2, power_spectrum_ch2 = welch(signal_ch2, fs=125e6 / decimation_factor, nperseg=1024)
        elif processing_method.lower() == 'fft':
            # Using FFT
            N = len(signal_ch2)
            frequencies_ch2 = np.fft.fftfreq(N, d=1/(125e6 / decimation_factor))
            power_spectrum_ch2 = np.abs(np.fft.fft(signal_ch2))**2 / N
        elif processing_method.lower() == 'bd':
            # Placeholder for Blind Deconvolution
            # Implement blind deconvolution here
            # For demonstration, we'll use the same as Welch
            frequencies_ch2, power_spectrum_ch2 = welch(signal_ch2, fs=125e6 / decimation_factor, nperseg=1024)
        else:
            raise ValueError(f"Invalid processing method: {processing_method}")
        
        # Avoid log of zero by replacing zeros with a very small number
        power_spectrum_ch2 = np.where(power_spectrum_ch2 == 0, 1e-12, power_spectrum_ch2)
        power_dbm_ch2 = 10 * np.log10(power_spectrum_ch2 / 50)  # Assuming 50 ohm impedance
        mask_ch2 = (frequencies_ch2 >= 10e3) & (frequencies_ch2 <= 100e3)
        filtered_power_dbm_ch2 = power_dbm_ch2[mask_ch2]
        features_ch2 = compute_statistical_features(filtered_power_dbm_ch2)
        
        # Add CH2 features to session data
        for key, value in features_ch2.items():
            session_data[f'ch2_{key}'] = value
        
    except Exception as e:
        print(f"Error processing session {ch1_filepath} and {ch2_filepath}: {e}")
        return None
    
    return session_data

# ---------------------------
# Main Processing Function
# ---------------------------

def main():
    # Load configuration
    try:
        config = load_config('config.json')
    except (FileNotFoundError, KeyError, ValueError) as e:
        print(f"Configuration Error: {e}")
        return
    
    base_dir = config['base_dir']
    device_label = config['device_label']
    processing_method = config['processing_method']
    decimation_factor = config.get('decimation_factor', 1024)  # Default to 1024 if not specified
    
    # Initialize a list to collect session data
    summary_data = []
    
    # Traverse the directory structure
    device_dir_path = os.path.join(base_dir, device_label)
    if not os.path.exists(device_dir_path):
        print(f"Device directory '{device_dir_path}' does not exist.")
        return
    
    # Iterate over date directories
    for date_dir in sorted(os.listdir(device_dir_path)):
        date_dir_path = os.path.join(device_dir_path, date_dir)
        if not os.path.isdir(date_dir_path):
            continue  # Skip if not a directory
        
        print(f"Processing date directory: {date_dir_path}")
        
        # Find all session_###_CH1.csv files
        session_pattern = re.compile(r'session_(\d{3})_CH1\.csv')
        for filename in sorted(os.listdir(date_dir_path)):
            match = session_pattern.match(filename)
            if match:
                session_number = match.group(1)
                ch1_filepath = os.path.join(date_dir_path, filename)
                ch2_filename = f'session_{session_number}_CH2.csv'
                ch2_filepath = os.path.join(date_dir_path, ch2_filename)
                
                # Check if CH2 file exists
                if not os.path.exists(ch2_filepath):
                    print(f"Missing CH2 file for session {session_number} in {date_dir_path}")
                    continue
                
                print(f"Processing session {session_number}")
                
                # Process the session
                session_data = process_session(ch1_filepath, ch2_filepath, decimation_factor, processing_method)
                if session_data:
                    session_data['device_label'] = device_label
                    summary_data.append(session_data)
                    print(f"Session {session_number} processed successfully.")
                else:
                    print(f"Failed to process session {session_number} in {date_dir_path}")
    
    # Convert the list of dictionaries to a DataFrame
    summary_df = pd.DataFrame(summary_data, columns=[
        'date', 'timestamp', 'session',
        'ch1_entropy', 'ch1_skewness', 'ch1_interquartile_range',
        'ch1_kurtosis', 'ch1_percentile_75', 'ch1_range',
        'ch1_maximum', 'ch1_median', 'ch1_percentile_90',
        'ch1_mean_absolute_deviation',
        'ch2_entropy', 'ch2_skewness', 'ch2_interquartile_range',
        'ch2_kurtosis', 'ch2_percentile_75', 'ch2_range',
        'ch2_maximum', 'ch2_median', 'ch2_percentile_90',
        'ch2_mean_absolute_deviation',
        'device_label'
    ])
    
    # Define the output summary CSV path
    output_summary_path = os.path.join(base_dir, device_label, f'summary_features_{processing_method.lower()}.csv')
    
    # Save the summary DataFrame to CSV
    try:
        summary_df.to_csv(output_summary_path, index=False)
        print(f"Summary CSV successfully saved to {output_summary_path}")
    except Exception as e:
        print(f"Failed to save summary CSV: {e}")

# ---------------------------
# Entry Point
# ---------------------------

if __name__ == "__main__":
    main()


Processing date directory: /Volumes/One Touch/data_scpi/empty/2024-12-02
Summary CSV successfully saved to /Volumes/One Touch/data_scpi/empty/summary_features_welch.csv


# all, signal processing type by type (fft, welch, histogram)

In [6]:
import os
import json
import pandas as pd
import numpy as np
import re
from scipy import stats
from scipy.signal import welch
#from PyEMD import EMD
import matplotlib.pyplot as plt  # For histogram computation (optional)

# ---------------------------
# Configuration and Setup
# ---------------------------

def load_config(config_path='config.json'):
    """
    Load configuration parameters from a JSON file.
    """
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Configuration file '{config_path}' not found.")
    
    with open(config_path, 'r') as f:
        config = json.load(f)
    
    required_keys = ['base_dir']
    for key in required_keys:
        if key not in config:
            raise KeyError(f"Missing required configuration key: '{key}'")
    
    return config

# ---------------------------
# Statistical Feature Computation
# ---------------------------

def compute_statistical_features(data_array):
    """
    Compute statistical features from a data array (e.g., power spectrum or histogram).
    """
    features = {}
    try:
        features['entropy'] = stats.entropy(np.abs(data_array))
        features['skewness'] = stats.skew(data_array)
        features['interquartile_range'] = np.percentile(data_array, 75) - np.percentile(data_array, 25)
        features['kurtosis'] = stats.kurtosis(data_array)
        features['percentile_75'] = np.percentile(data_array, 75)
        features['range'] = np.ptp(data_array)
        features['maximum'] = np.max(data_array)
        features['median'] = np.median(data_array)
        features['percentile_90'] = np.percentile(data_array, 90)
        features['mean_absolute_deviation'] = np.mean(np.abs(data_array - np.mean(data_array)))
    except Exception as e:
        print(f"Error computing statistical features: {e}")
        features = {key: np.nan for key in ['entropy', 'skewness', 'interquartile_range', 'kurtosis',
                                            'percentile_75', 'range', 'maximum', 'median',
                                            'percentile_90', 'mean_absolute_deviation']}
    return features

# ---------------------------
# Channel Processing
# ---------------------------

def process_channel(df_channel, channel_label, decimation_factor, processing_method):
    """
    Process a single channel and return computed features.
    """
    try:
        if df_channel.empty or 'voltage' not in df_channel.columns:
            print(f"No data or missing 'voltage' column in {channel_label}")
            return None
        
        # Handle NaN values
        if df_channel['voltage'].isnull().any():
            print(f"NaN values found in {channel_label}. They will be ignored.")
            df_channel = df_channel.dropna(subset=['voltage'])
            if df_channel.empty:
                print(f"All data is NaN in {channel_label}. Skipping.")
                return None
        
        # Process the signal
        signal = df_channel['voltage'].values
        signal = signal - np.mean(signal)  # Remove DC component

        if processing_method.lower() == 'welch':
            frequencies, power_spectrum = welch(signal, fs=125e6 / decimation_factor, nperseg=1024)
            data_to_analyze = power_spectrum
        elif processing_method.lower() == 'fft':
            # Using FFT
            N = len(signal)
            frequencies = np.fft.fftfreq(N, d=1/(125e6 / decimation_factor))
            power_spectrum = np.abs(np.fft.fft(signal))**2 / N
            data_to_analyze = power_spectrum
        elif processing_method.lower() == 'histogram':
            # Compute histogram of the time-domain voltage signal
            hist_counts, bin_edges = np.histogram(signal, bins=50, density=True)
            data_to_analyze = hist_counts  # Use histogram counts for feature computation
        else:
            raise ValueError(f"Invalid processing method: {processing_method}")
        
        # For methods other than 'histogram', process power spectrum
        if processing_method.lower() != 'histogram':
            # Avoid log of zero by replacing zeros with a very small number
            data_to_analyze = np.where(data_to_analyze == 0, 1e-12, data_to_analyze)
            power_dbm = 10 * np.log10(data_to_analyze / 50)  # Assuming 50 ohm impedance
            mask = (frequencies >= 10e3) & (frequencies <= 100e3)
            filtered_data = power_dbm[mask]
        else:
            # For 'histogram', use the histogram counts directly
            filtered_data = data_to_analyze  # Already processed
        
        # Compute statistical features
        features = compute_statistical_features(filtered_data)
        
        # Prefix feature names with channel label
        features = {f"{channel_label}_{key}": value for key, value in features.items()}
        
        return features
    
    except Exception as e:
        print(f"Error processing {channel_label}: {e}")
        return None

# ---------------------------
# Session Processing
# ---------------------------

def process_session(ch1_filepath, ch2_filepath, decimation_factor, processing_method):
    """
    Process a single session by computing features for CH1 and CH2 using the specified processing method.
    Returns a dictionary with all required information.
    """
    session_data = {}
    try:
        # Extract session number from filename
        session_match = re.search(r'session_(\d{3})_CH1\.csv', os.path.basename(ch1_filepath))
        if not session_match:
            print(f"Invalid CH1 filename format: {ch1_filepath}")
            return None
        session_number = int(session_match.group(1))
        session_data['session'] = session_number
        
        # Extract date from filepath
        # Assuming the path is base_dir/device_label/YYYY-MM-DD/session_###_CHx.csv
        date_str = os.path.basename(os.path.dirname(ch1_filepath))
        session_data['date'] = date_str
        
        # Read CH1 data
        df_ch1 = pd.read_csv(ch1_filepath)
        
        # Process CH1
        ch1_features = process_channel(df_ch1, 'ch1', decimation_factor, processing_method)
        if ch1_features is None:
            print(f"Failed to process CH1 for session {session_number}")
            return None
        
        # Get the first timestamp as session timestamp
        timestamp_str = df_ch1['timestamp'].iloc[0]
        session_data['timestamp'] = timestamp_str
        
        # Read CH2 data
        df_ch2 = pd.read_csv(ch2_filepath)
        
        # Process CH2
        ch2_features = process_channel(df_ch2, 'ch2', decimation_factor, processing_method)
        if ch2_features is None:
            print(f"Failed to process CH2 for session {session_number}")
            return None
        
        # Combine features
        session_data.update(ch1_features)
        session_data.update(ch2_features)
        
    except Exception as e:
        print(f"Error processing session {ch1_filepath} and {ch2_filepath}: {e}")
        return None
    
    return session_data

# ---------------------------
# Main Processing Function
# ---------------------------

def main():
    # Load configuration
    try:
        config = load_config('config.json')
    except (FileNotFoundError, KeyError, ValueError) as e:
        print(f"Configuration Error: {e}")
        return
    
    base_dir = config['base_dir']
    decimation_factor = config.get('decimation_factor', 1024)  # Default to 1024 if not specified
    # processing_methods = ['fft', 'welch', 'bd', 'histogram']  # List of processing methods
    processing_methods = ['welch']

    # Get list of devices in base_dir
    device_labels = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

    for device_label in device_labels:
        device_dir_path = os.path.join(base_dir, device_label)
        print(f"\nProcessing device: {device_label}")
        
        for processing_method in processing_methods:
            print(f"\nUsing processing method: {processing_method.upper()}")
            # Initialize a list to collect session data
            summary_data = []

            # Iterate over date directories
            date_dirs = [d for d in os.listdir(device_dir_path) if os.path.isdir(os.path.join(device_dir_path, d))]
            for date_dir in sorted(date_dirs):
                date_dir_path = os.path.join(device_dir_path, date_dir)
                
                print(f"Processing date directory: {date_dir_path}")
                
                # Find all session_###_CH1.csv files
                session_pattern = re.compile(r'session_(\d{3})_CH1\.csv')
                for filename in sorted(os.listdir(date_dir_path)):
                    match = session_pattern.match(filename)
                    if match:
                        session_number = match.group(1)
                        ch1_filepath = os.path.join(date_dir_path, filename)
                        ch2_filename = f'session_{session_number}_CH2.csv'
                        ch2_filepath = os.path.join(date_dir_path, ch2_filename)
                        
                        # Check if CH2 file exists
                        if not os.path.exists(ch2_filepath):
                            print(f"Missing CH2 file for session {session_number} in {date_dir_path}")
                            continue
                        
                        print(f"Processing session {session_number}")
                        
                        # Process the session
                        session_data = process_session(ch1_filepath, ch2_filepath, decimation_factor, processing_method)
                        if session_data:
                            session_data['device_label'] = device_label
                            summary_data.append(session_data)
                            print(f"Session {session_number} processed successfully.")
                        else:
                            print(f"Failed to process session {session_number} in {date_dir_path}")

            if summary_data:
                # Convert the list of dictionaries to a DataFrame
                summary_df = pd.DataFrame(summary_data)
                
                # Define the output summary CSV path
                output_summary_path = os.path.join(base_dir, device_label, f'summary_features_{processing_method.lower()}.csv')
                
                # Save the summary DataFrame to CSV
                try:
                    summary_df.to_csv(output_summary_path, index=False)
                    print(f"Summary CSV successfully saved to {output_summary_path}")
                except Exception as e:
                    print(f"Failed to save summary CSV: {e}")
            else:
                print(f"No data processed for device '{device_label}' with method '{processing_method}'.")

# ---------------------------
# Entry Point
# ---------------------------

if __name__ == "__main__":
    main()



Processing device: empty

Using processing method: WELCH
Processing date directory: /Volumes/One Touch/data_scpi/empty/2024-12-02
No data processed for device 'empty' with method 'welch'.

Processing device: lamp

Using processing method: WELCH
Processing date directory: /Volumes/One Touch/data_scpi/lamp/2024-12-02
No data processed for device 'lamp' with method 'welch'.


# LATEST processing

In [3]:
import os
import json
import pandas as pd
import numpy as np
import re
from scipy import stats
from scipy.signal import welch
import matplotlib.pyplot as plt  # For histogram computation (optional)

# ---------------------------
# Configuration and Setup
# ---------------------------

def load_config(config_path='config.json'):
    """
    Load configuration parameters from a JSON file.
    """
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Configuration file '{config_path}' not found.")
    
    with open(config_path, 'r') as f:
        config = json.load(f)
    
    required_keys = ['base_dir']
    for key in required_keys:
        if key not in config:
            raise KeyError(f"Missing required configuration key: '{key}'")
    
    return config

# ---------------------------
# Function to Convert Binary Data to DataFrame
# ---------------------------

def binary_to_dataframe(binary_filepath, input_range=1.0, adc_bits=16):
    """
    Convert a binary file containing interleaved CH1 and CH2 data into a DataFrame.

    Parameters:
        binary_filepath (str): Path to the binary file.
        input_range (float): Input range of the ADC (1:1 divider).
        adc_bits (int): Number of ADC bits

    Returns:
        pd.DataFrame: DataFrame containing voltage_ch1 and voltage_ch2.
    """
    # Read the binary file
    with open(binary_filepath, 'rb') as f:
        raw_data = f.read()
    
    # Convert to 16-bit integers
    samples = np.frombuffer(raw_data, dtype=np.int16)
    
    # Split interleaved data
    ch1_samples = samples[::2]  # Every other sample for CH1
    ch2_samples = samples[1::2]  # Every other sample for CH2

    # Convert raw ADC values to voltages
    voltage_ch1 = np.array(ch1_samples) * (input_range / (2**(adc_bits - 1)))
    voltage_ch2 = np.array(ch2_samples) * (input_range / (2**(adc_bits - 1)))
    
    # Create a DataFrame
    df = pd.DataFrame({
        'voltage_ch1': voltage_ch1,
        'voltage_ch2': voltage_ch2
    })
    
    return df

# ---------------------------
# Statistical Feature Computation
# ---------------------------

def compute_statistical_features(data_array):
    """
    Compute statistical features from a data array (e.g., power spectrum or histogram).
    """
    features = {}
    try:
        features['entropy'] = stats.entropy(np.abs(data_array))
        features['skewness'] = stats.skew(data_array)
        features['interquartile_range'] = np.percentile(data_array, 75) - np.percentile(data_array, 25)
        features['kurtosis'] = stats.kurtosis(data_array)
        features['percentile_75'] = np.percentile(data_array, 75)
        features['range'] = np.ptp(data_array)
        features['maximum'] = np.max(data_array)
        features['median'] = np.median(data_array)
        features['percentile_90'] = np.percentile(data_array, 90)
        features['mean_absolute_deviation'] = np.mean(np.abs(data_array - np.mean(data_array)))
    except Exception as e:
        print(f"Error computing statistical features: {e}")
        features = {key: np.nan for key in ['entropy', 'skewness', 'interquartile_range', 'kurtosis',
                                            'percentile_75', 'range', 'maximum', 'median',
                                            'percentile_90', 'mean_absolute_deviation']}
    return features

# ---------------------------
# Welch Processing Function
# ---------------------------

def process_welch(signal, sampling_rate):
    """
    Process the signal using Welch's method and compute the power spectrum.

    Parameters:
        signal (np.ndarray): The input signal.
        sampling_rate (float): The sampling rate of the signal.

    Returns:
        np.ndarray: Filtered power spectrum in dBm.
    """
    frequencies, power_spectrum = welch(signal, fs=sampling_rate, nperseg=1024)
    # Avoid log of zero by replacing zeros with a very small number
    power_spectrum = np.where(power_spectrum == 0, 1e-12, power_spectrum)
    power_dbm = 10 * np.log10(power_spectrum / 50)  # Assuming 50 ohm impedance
    mask = (frequencies >= 10e3) & (frequencies <= 100e3)
    filtered_data = power_dbm[mask]
    return filtered_data

# ---------------------------
# Histogram Processing Function
# ---------------------------

def process_histogram(signal):
    """
    Compute the histogram of the signal.

    Parameters:
        signal (np.ndarray): The input signal.

    Returns:
        np.ndarray: Histogram counts for feature computation.
    """
    hist_counts, bin_edges = np.histogram(signal, bins=50, density=True)
    data_to_analyze = hist_counts  # Use histogram counts for feature computation
    return data_to_analyze

# ---------------------------
# Channel Processing
# ---------------------------

def process_channel(signal, channel_label, decimation_factor, processing_method):
    """
    Process a single channel and return computed features.
    """
    try:
        if signal.size == 0:
            print(f"No data in {channel_label}")
            return None
        
        # Remove DC component
        signal = signal - np.mean(signal)

        sampling_rate = 125e6 / decimation_factor

        if processing_method.lower() == 'welch':
            filtered_data = process_welch(signal, sampling_rate)
        elif processing_method.lower() == 'hist':
            filtered_data = process_histogram(signal)
        else:
            raise ValueError(f"Invalid processing method: {processing_method}")

        # Compute statistical features
        features = compute_statistical_features(filtered_data)
        
        # Prefix feature names with channel label
        features = {f"{channel_label}_{key}": value for key, value in features.items()}
        
        return features

    except Exception as e:
        print(f"Error processing {channel_label}: {e}")
        return None

# ---------------------------
# Data Augmentation Function
# ---------------------------

def augment_signal(signal, augmentation_method='overlap', overlap_ratio=0.5):
    """
    Augment the signal using specified method.

    Parameters:
        signal (np.ndarray): The input signal.
        augmentation_method (str): The augmentation method to apply.
        overlap_ratio (float): The ratio of overlap between chunks.

    Returns:
        List[np.ndarray]: List of augmented signal chunks.
    """
    augmented_signals = []
    if augmentation_method == 'overlap':
        chunk_size = 10000  # Same as batch_size
        step_size = int(chunk_size * (1 - overlap_ratio))
        for start_idx in range(0, len(signal) - chunk_size + 1, step_size):
            end_idx = start_idx + chunk_size
            augmented_signals.append(signal[start_idx:end_idx])
    else:
        # Implement other augmentation methods if needed
        augmented_signals.append(signal)
    return augmented_signals

# ---------------------------
# Session Processing
# ---------------------------

def process_session(binary_filepath, decimation_factor, batch_size=10000, augment=False):
    """
    Process a single session by computing features for CH1 and CH2 in batches.

    Parameters:
        binary_filepath (str): Path to the binary file.
        decimation_factor (int): Decimation factor for signal processing.
        batch_size (int): Number of samples per batch.
        augment (bool): Whether to augment data using overlapping chunks.

    Returns:
        List[Dict]: List of dictionaries containing features for each batch.
    """
    session_data_list = []
    try:
        # Extract session number from filename
        session_match = re.search(r'session_(\d{3})\.bin', os.path.basename(binary_filepath))
        if not session_match:
            print(f"Invalid session filename format: {binary_filepath}")
            return None
        session_number = int(session_match.group(1))
        
        # Extract date from filepath
        date_str = os.path.basename(os.path.dirname(binary_filepath))
        
        # Read binary file into DataFrame
        df = binary_to_dataframe(binary_filepath)
        
        # Total number of samples
        total_samples = len(df)
        
        # If total_samples is less than batch_size, skip into the next session
        if total_samples < batch_size:
            print(f"Session {session_number} has less than {batch_size} samples. Skipping to next session.")
            return None
        
        # Determine the number of batches
        if augment:
            step_size = int(batch_size * 0.5)  # 50% overlap
            num_batches = (total_samples - batch_size) // step_size + 1
        else:
            num_batches = total_samples // batch_size
            step_size = batch_size

        for batch_index in range(num_batches):
            start_idx = batch_index * step_size
            end_idx = start_idx + batch_size

            # Skip if we don't have enough samples
            if end_idx > total_samples:
                break

            batch_df = df.iloc[start_idx:end_idx]

            batch_data = {}
            batch_data['session'] = session_number
            batch_data['date'] = date_str
            batch_data['batch'] = batch_index

            # Prepare data for CH1 and CH2
            ch1_signal = batch_df['voltage_ch1'].values
            ch2_signal = batch_df['voltage_ch2'].values

            # For each processing method (Welch and Histogram)
            for processing_method in ['welch', 'hist']:

                # Process CH1
                ch1_features = process_channel(ch1_signal, 'ch1', decimation_factor, processing_method)
                if ch1_features is None:
                    print(f"Failed to process CH1 for batch {batch_index} in session {session_number}")
                    continue

                # Process CH2
                ch2_features = process_channel(ch2_signal, 'ch2', decimation_factor, processing_method)
                if ch2_features is None:
                    print(f"Failed to process CH2 for batch {batch_index} in session {session_number}")
                    continue

                # Add features to batch_data with processing method suffix
                for key, value in ch1_features.items():
                    batch_data[f"{key}_{processing_method}"] = value

                for key, value in ch2_features.items():
                    batch_data[f"{key}_{processing_method}"] = value

            # Append batch_data to session_data_list
            session_data_list.append(batch_data)

    except Exception as e:
        print(f"Error processing session {binary_filepath}: {e}")
        return None

    return session_data_list

# ---------------------------
# Main Processing Function
# ---------------------------

def main():
    # Load configuration
    try:
        config = load_config('config.json')
    except (FileNotFoundError, KeyError, ValueError) as e:
        print(f"Configuration Error: {e}")
        return

    base_dir = config['base_dir']
    decimation_factor = config.get('decimation_factor', 1024)  # Default to 1024 if not specified
    batch_size = 10000

    # Prompt user for data augmentation
    augment_choice = input("Do you want to augment the data with overlapping chunks? (yes/no): ").strip().lower()
    if augment_choice in ['yes', 'y']:
        augment = True
        output_filename = 'summary_features_augmented.csv'
        print("Data augmentation enabled. Using overlapping chunks.")
    else:
        augment = False
        output_filename = 'summary_features.csv'
        print("Data augmentation disabled.")

    # Get list of devices in base_dir
    device_labels = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

    for device_label in device_labels:
        device_dir_path = os.path.join(base_dir, device_label)
        print(f"\nProcessing device: {device_label}")

        # Initialize a list to collect session data
        summary_data = []

        # Iterate over date directories
        date_dirs = [d for d in os.listdir(device_dir_path) if os.path.isdir(os.path.join(device_dir_path, d))]
        for date_dir in sorted(date_dirs):
            date_dir_path = os.path.join(device_dir_path, date_dir)

            print(f"Processing date directory: {date_dir_path}")

            # Find all session_###.bin files
            session_pattern = re.compile(r'session_(\d{3})\.bin')
            for filename in sorted(os.listdir(date_dir_path)):
                match = session_pattern.match(filename)
                if match:
                    session_number = match.group(1)
                    binary_filepath = os.path.join(date_dir_path, filename)

                    print(f"Processing session {session_number}")

                    # Process the session
                    session_data_list = process_session(binary_filepath, decimation_factor, batch_size, augment=augment)
                    if session_data_list:
                        for batch_data in session_data_list:
                            batch_data['device_label'] = device_label
                            summary_data.append(batch_data)
                        print(f"Session {session_number} processed successfully.")
                    else:
                        print(f"Failed to process session {session_number} in {date_dir_path}")

        if summary_data:
            # Convert the list of dictionaries to a DataFrame
            summary_df = pd.DataFrame(summary_data)

            # Define the output summary CSV path
            output_summary_path = os.path.join(base_dir, device_label, output_filename)

            # Save the summary DataFrame to CSV
            try:
                summary_df.to_csv(output_summary_path, index=False)
                print(f"Summary CSV successfully saved to {output_summary_path}")
            except Exception as e:
                print(f"Failed to save summary CSV: {e}")
        else:
            print(f"No data processed for device '{device_label}'.")

# ---------------------------
# Entry Point
# ---------------------------

if __name__ == "__main__":
    main()


Data augmentation disabled.

Processing device: empty
Processing date directory: /Volumes/One Touch/data_test/empty/2024-11-23
Processing session 001
Session 001 processed successfully.
Processing session 002
Session 002 processed successfully.
Processing session 003
Session 003 processed successfully.
Processing session 004
Session 004 processed successfully.
Processing session 005
Session 005 processed successfully.
Processing session 006
Session 006 processed successfully.
Processing session 007
Session 007 processed successfully.
Processing session 008
Session 008 processed successfully.
Processing session 009
Session 009 processed successfully.
Processing session 010
Session 010 processed successfully.
Summary CSV successfully saved to /Volumes/One Touch/data_test/empty/summary_features.csv
